# Installs

In [1]:
!pip install -q tqdm
!pip install -q seaborn
!pip install -q datasets
!pip install -q scikit-learn
!pip install -q pytorch_lightning
!pip install -q git+https://github.com/MagedSaeed/tkseem

# Prepare

In [1]:
import re
import os
import shutil
from pathlib import Path

from pyarabic import araby

from sklearn.model_selection import train_test_split

import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import torchmetrics
from torchmetrics.functional import word_error_rate, char_error_rate

from pytorch_lightning import seed_everything
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import LightningModule,Trainer
from pytorch_lightning.callbacks import EarlyStopping,LearningRateMonitor,ModelCheckpoint,RichProgressBar

from dotless_arabic.datasets.wikipedia.collect import collect_dataset_for_dots_retreival
from dotless_arabic import constants
from dotless_arabic.processing import undot,process

import datasets
import seaborn as sns
import matplotlib.pyplot as plt


import tkseem as tk
from tqdm.auto import tqdm

In [2]:
seed = 42

In [3]:
# random.seed(seed)     # python random generator
# np.random.seed(seed)  # numpy random generator

# torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)

# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

seed_everything(seed)

Global seed set to 42


42

In [4]:
tqdm.pandas()

# Load and explore the dataset

In [5]:
dataset = list(set(collect_dataset_for_dots_retreival()))
len(dataset)

  0%|          | 0/4636663 [00:00<?, ?it/s]

  0%|          | 0/4636645 [00:00<?, ?it/s]

  0%|          | 0/10867699 [00:00<?, ?it/s]

1440782

In [6]:
train_dataset,test_dataset = train_test_split(dataset,test_size=0.05,shuffle=True)
len(train_dataset),len(test_dataset)

(1368742, 72040)

trainset vocabulary and tokens count:


In [7]:
vocabs_dict = {}
for document in tqdm(train_dataset):
  for word in document.split():
    vocabs_dict[word] = vocabs_dict.get(word,0)+1
f'{len(vocabs_dict.keys()):,}',f'{sum(vocabs_dict.values()):,}'

  0%|          | 0/1368742 [00:00<?, ?it/s]

('2,552,981', '61,515,015')

# Clean and Preprocess the dataset

In [8]:
# text cleaning

def rm_link(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

# handle case like "shut up okay?Im only 10 years old"
# become "shut up okay Im only 10 years old"
def rm_punct2(text):
    return re.sub(r'[\!\"\#\$\%\&\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~]', ' ', text)
    # return re.sub(r'[\"\#\$\%\&\'\(\)\*\+\/\:\;\<\=\>\@\[\\\]\^\_\`\{\|\}\~]', ' ', text)
    # return text.translate(str.maketrans('', '', string.punctuation))

def rm_html(text):
    return re.sub(r'<[^>]+>', '', text)

def space_bt_punct(text):
    pattern = r'([.,!?-])'
    s = re.sub(pattern, r' \1 ', text)     # add whitespaces between punctuation
    s = re.sub(r'\s{2,}', ' ', s)        # remove double whitespaces
    return s

def rm_number(text):
    return re.sub(r'\d+', '', text)

def rm_extra_whitespaces(text):
    return re.sub(r' +', ' ', text)

# def rm_nonascii(text):
#     return re.sub(r'[^\x00-\x7f]', r'', text)

def rm_emoji(text):
    emojis = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE
    )
    return emojis.sub(r'', text)

# def spell_correction(text):
#     return re.sub(r'(.)\1+', r'\1\1', text)

def clean_pipeline(text):
    text = rm_link(text)
    text = rm_html(text)
    text = space_bt_punct(text)
    # text = rm_punct2(text)
    text = rm_number(text)
    # text = rm_nonascii(text)
    text = rm_emoji(text)
    """arabic specific"""
    text = araby.strip_diacritics(text)
    text = araby.strip_tatweel(text)
    text = araby.normalize_alef(text)
    text = araby.normalize_hamza(text)
    text = araby.normalize_ligature(text)
    # remove any non arabic character
    text = "".join(
        [c for c in text if c in constants.ARABIC_LETTERS or c.isspace()]
    )
    text = rm_extra_whitespaces(text)
    # text = spell_correction(text)
    return process(text).strip()

In [9]:
clean_pipeline('السلام عليكم ورحمة. الله')

'السلام عليكم ورحمة الله'

In [10]:
def prepare(text):
  return clean_pipeline(text)

In [11]:
# test the prepare method
prepare('hello بالإنجليزية تعني أهلاً')

'بالانجليزية تعني اهلا'

In [12]:
train_dataset = list(map(prepare,tqdm(train_dataset)))
train_dataset = list(filter(lambda doc:len(doc)>0,tqdm(train_dataset)))
train_dataset[:2]

  0%|          | 0/1368742 [00:00<?, ?it/s]

  0%|          | 0/1368742 [00:00<?, ?it/s]

['عاش مواطنو كوريا الجنوبية تحت انظمة حكم غير ديموقراطية معظم القرن العشرين وهي انظمة شمولية عسكرية لزعماء امثال اي سنغ مان وباك تشونغ هي وتشون دو هوان وروو تاي وو',
 'وعندما دخل السفير الاسراءيلي الياهو بن اليسار الا قاعة العرش وقدم اوراق اعتماده سفيرا فوق العادة ومطلق الصلاحية الا الملك محمد انور بن فاروق بن فءاد بن السادات توقفنا عن اذاعة القران الكريم ونصبنا الصلوات وبدانا نستقبل المعزين']

In [13]:
test_dataset = list(map(prepare,tqdm(test_dataset)))
test_dataset = list(filter(lambda doc:len(doc)>0,tqdm(test_dataset)))
test_dataset[:2]

  0%|          | 0/72040 [00:00<?, ?it/s]

  0%|          | 0/72040 [00:00<?, ?it/s]

['في حزيران غادر ليون بلوم منصبه وعاد ادوار دالادير الا الحكم رافعا شعار التمسك بالامبراطورية الفرنسية فسافر الرءيس جميل مردم بك الا باريس في تشرين الثاني ووقع عدة ملاحق للمعاهدة السورية الفرنسية فيها الكثير من التنازلات منها ضمانات اضافية للاقليات والتاكيد علا الاستعانة بخبراء فرنسيين داءمين في مءسسات الحكم السورية',
 'يعتبر الحكم سعد العقيل من الحكام السابقين في التحكيم ولقد عاصر كثيرا من الحكام مثل الحكم عمر المهنا وعبد الله الناصر وفلاج الشنار والشريف وعبد الرحمن الدهام وفهد الدهمش وكثير من الحكام ولكن ظلم هذا الحكم من الاعلام ويعتبر من اشهر الحكام في منطقة الاحساء علا زمان الحكم حسين الحساوي وسعد ربيعة']

In [14]:
vocabs_dict = {}
for document in tqdm(train_dataset):
  for word in document.split():
    vocabs_dict[word] = vocabs_dict.get(word,0)+1
f'{len(vocabs_dict.keys()):,}',f'{sum(vocabs_dict.values()):,}'

  0%|          | 0/1368224 [00:00<?, ?it/s]

('1,040,339', '59,208,516')

# Helper functions and constants

In [15]:
train_dataset[0]

'عاش مواطنو كوريا الجنوبية تحت انظمة حكم غير ديموقراطية معظم القرن العشرين وهي انظمة شمولية عسكرية لزعماء امثال اي سنغ مان وباك تشونغ هي وتشون دو هوان وروو تاي وو'

In [16]:
# Find out the max samples token
# sorted_docs_by_length = sorted(tqdm(train_dataset),key=lambda document: len(document.split()),reverse=True)
sorted_docs_by_length = sorted(tqdm(train_dataset),key=len,reverse=True)
len(sorted_docs_by_length[0]),\
len(sorted_docs_by_length[1]),\
len(sorted_docs_by_length[2]),\
len(sorted_docs_by_length[5]),\
len(sorted_docs_by_length[10]),\
len(sorted_docs_by_length[50]),\
len(sorted_docs_by_length[1_000]),\
len(sorted_docs_by_length[2_500]),\
len(sorted_docs_by_length[5_000]),\
len(sorted_docs_by_length[10_000])

  0%|          | 0/1368224 [00:00<?, ?it/s]

(8115, 7783, 7723, 6226, 4763, 2955, 1431, 1127, 940, 786)

In [17]:
# Find out the max samples token
# sorted_docs_by_length = sorted(tqdm(test_dataset),key=lambda document: len(document.split()),reverse=True)
sorted_docs_by_length = sorted(tqdm(test_dataset),key=len,reverse=True)
len(sorted_docs_by_length[0]),\
len(sorted_docs_by_length[1]),\
len(sorted_docs_by_length[2]),\
len(sorted_docs_by_length[5]),\
len(sorted_docs_by_length[10]),\
len(sorted_docs_by_length[50]),\
len(sorted_docs_by_length[1_000]),\
len(sorted_docs_by_length[2_500]),\
len(sorted_docs_by_length[5_000]),\
len(sorted_docs_by_length[10_000])

  0%|          | 0/72025 [00:00<?, ?it/s]

(3542, 3070, 3065, 2613, 2338, 1489, 661, 511, 412, 332)

In [18]:
# setting seq_len:
seq_len = 500

In [19]:
def create_features_from_text_list(text_list,tokenizer):
  encoded = list()
  for doc in tqdm(text_list):
    encoded_doc = tokenizer.encode(doc)
    encoded_doc = tokenizer.pad(encoded_doc,length=seq_len)
    encoded_doc = encoded_doc[:seq_len]
    encoded.append(np.array(encoded_doc))
  return np.array(encoded)

In [20]:
# define batch size
batch_size = 128

In [90]:
def calculate_text_metrics(predictions,labels,target_tokenizer, print_text=False):
  # drop pads, those pads are not necessary pad tokens!!
  last_pad = predictions[-1]
  for i,pad in reversed(list(enumerate(predictions))):
    if pad == last_pad:
      predictions.pop(i)
    else:
      break

  predicted_text = target_tokenizer.detokenize(target_tokenizer.decode(predictions))
  predicted_text = predicted_text.replace('<PAD>','').strip()

  true_text = target_tokenizer.detokenize(target_tokenizer.decode(labels))
  true_text = true_text.replace('<PAD>','').strip()

  if print_text:
    print('*'*100)
    print(predicted_text)
    print(true_text)

  wer = word_error_rate(preds=predicted_text, target=true_text)
  cer = char_error_rate(preds=predicted_text, target=true_text)
  return wer,cer,predicted_text,true_text

# Add masked consontants column to the dataset

In [22]:
undotted_train_dataset = list(map(undot,tqdm(train_dataset)))
undotted_train_dataset[:2]

  0%|          | 0/1368224 [00:00<?, ?it/s]

['عاس مواطٮو كورٮا الحٮوٮٮه ٮحٮ اٮطمه حكم عٮر دٮموڡراطٮه معطم الڡرں العسرٮں وهى اٮطمه سمولٮه عسكرٮه لرعماء امٮال اى سٮع ماں وٮاك ٮسوٮع هى وٮسوں دو هواں وروو ٮاى وو',
 'وعٮدما دحل السڡٮر الاسراءٮلى الٮاهو ٮں الٮسار الا ڡاعه العرس وڡدم اوراٯ اعٮماده سڡٮرا ڡوٯ العاده ومطلٯ الصلاحٮه الا الملك محمد اٮور ٮں ڡاروٯ ٮں ڡءاد ٮں الساداٮ ٮوڡڡٮا عں اداعه الڡراں الكرٮم وٮصٮٮا الصلواٮ وٮداٮا ٮسٮڡٮل المعرٮں']

In [23]:
undotted_test_dataset = list(map(undot,tqdm(test_dataset)))
undotted_test_dataset[:2]

  0%|          | 0/72025 [00:00<?, ?it/s]

['ڡى حرٮراں عادر لٮوں ٮلوم مٮصٮه وعاد ادوار دالادٮر الا الحكم راڡعا سعار الٮمسك ٮالامٮراطورٮه الڡرٮسٮه ڡساڡر الرءٮس حمٮل مردم ٮك الا ٮارٮس ڡى ٮسرٮں الٮاٮى ووڡع عده ملاحٯ للمعاهده السورٮه الڡرٮسٮه ڡٮها الكٮٮر مں الٮٮارلاٮ مٮها صماٮاٮ اصاڡٮه للاڡلٮاٮ والٮاكٮد علا الاسٮعاٮه ٮحٮراء ڡرٮسٮٮں داءمٮں ڡى مءسساٮ الحكم السورٮه',
 'ٮعٮٮر الحكم سعد العڡٮل مں الحكام الساٮڡٮں ڡى الٮحكٮم ولڡد عاصر كٮٮرا مں الحكام مٮل الحكم عمر المهٮا وعٮد الله الٮاصر وڡلاح السٮار والسرٮڡ وعٮد الرحمں الدهام وڡهد الدهمس وكٮٮر مں الحكام ولكں طلم هدا الحكم مں الاعلام وٮعٮٮر مں اسهر الحكام ڡى مٮطڡه الاحساء علا رماں الحكم حسٮں الحساوى وسعد رٮٮعه']

# Build the BiLSTM Model

In [24]:
# model architecture (with some amends)
# https://www.kaggle.com/code/affand20/imdb-with-pytorch

class LitBiLSTMModel(LightningModule):
    def __init__(
        self,
        vocab_size,
        output_size,
        pad_id=1, # the default value from tkseem tokenizers
        seq_len=seq_len,
        hidden_size=512,
        embedding_size=512,
        embedding_dropout=0.25,
        dropout=0.33,
        learning_rate=0.001,
        n_layers=2,
        bidirectional=True,
      ):
        super().__init__()

        self.save_hyperparameters()

        self.bidirectional = bidirectional
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate
        self.dropout_prop = dropout
        self.pad_id = pad_id
        self.max_sequence_length = seq_len
        self.output_size = output_size

        self.train_accuracy = torchmetrics.Accuracy(
            task="multiclass",
            num_classes=output_size,
            ignore_index=self.pad_id,
          )
        self.val_accuracy = torchmetrics.Accuracy(
            task="multiclass",
            num_classes=output_size,
            ignore_index=self.pad_id,
          )
        self.test_accuracy = torchmetrics.Accuracy(
            task="multiclass",
            num_classes=output_size,
            ignore_index=self.pad_id,
          )

        # embedding layer is useful to map input into vector representation
        self.embedding = nn.Embedding(vocab_size, embedding_size)

        # LSTM layer preserved by PyTorch library
        self.lstm = nn.LSTM(
              embedding_size,
              hidden_size,
              n_layers,
              dropout=dropout,
              batch_first=True,
              bidirectional=bidirectional,
        )

        # dropout layer
        self.dropout = nn.Dropout(dropout)
        self.embedding_dropout = nn.Dropout(embedding_dropout)

        # Linear layer for output
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, inputs):
        outputs = self.embedding(inputs)
        outputs = self.embedding_dropout(outputs) # apply dropout on embedding
        inputs_lengths = torch.sum(inputs!=self.pad_id,axis=-1).cpu()
        packed_outputs = nn.utils.rnn.pack_padded_sequence(
            outputs,
            inputs_lengths,
            batch_first=True,
            enforce_sorted=False,
          )
        if not self.bidirectional:
          # pass forward to lstm
          packed_outputs, _ =  self.lstm(packed_outputs)
          outputs,lengths = nn.utils.rnn.pad_packed_sequence(
              packed_outputs,
              batch_first=True,
              total_length=self.max_sequence_length,
            )
        else:
          bidirectional_packed_outputs,_ = self.lstm(packed_outputs)
          bidirectional_outputs,lengths = nn.utils.rnn.pad_packed_sequence(
              bidirectional_packed_outputs,
              batch_first=True,
              # padding_value=self.pad_id,
              total_length=self.max_sequence_length,
            )
          outputs = bidirectional_outputs[:, :, : self.hidden_size] + bidirectional_outputs[:, :, self.hidden_size :]
        outputs = self.dropout(outputs)
        outputs = self.fc(outputs)
        # softmax will be done in the loss calculation
        return outputs

    def step(self, inputs, labels):
        assert torch.sum(inputs==self.pad_id)==torch.sum(labels==self.pad_id),\
             f'pad ids and their target tags does not match: {torch.sum(inputs==self.pad_id):=} != {torch.sum(labels==self.pad_id):=}'
        outputs = self(inputs)
        outputs = outputs.squeeze()
        outputs = outputs.view(outputs.shape[0]*outputs.shape[1],-1)
        labels = labels.view(-1)
        return outputs,labels

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs,labels = self.step(inputs, labels)
        loss = F.cross_entropy(
            outputs,
            labels,
            ignore_index=self.pad_id,
          )
        train_accuracy = self.train_accuracy(outputs, labels)
        self.log(
            "loss",
            loss,
            prog_bar=True,
            on_step=True,
            on_epoch=False,
        )
        self.log(
            "train_acc",
            train_accuracy,
            on_step=True,
            on_epoch=False,
            prog_bar=True,
            logger=True,
        )
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs,labels = self.step(inputs, labels)
        loss = F.cross_entropy(
            outputs,
            labels,
            ignore_index=self.pad_id,
          )
        val_accuracy = self.val_accuracy(outputs, labels)
        self.log("val_loss", loss, prog_bar=True)
        self.log(
            "val_acc",
            val_accuracy,
            on_step=True,
            on_epoch=False,
            prog_bar=True,
            logger=True,
        )
        return {"val_loss": loss}

    def test_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs,labels = self.step(inputs, labels)
        loss = F.cross_entropy(
            outputs,
            labels,
            ignore_index=self.pad_id,
          )
        test_accuracy = self.test_accuracy(outputs,labels)
        metrics = {"test_acc": test_accuracy, "test_loss": loss}
        self.log_dict(metrics, prog_bar=True)
        return outputs

    def predict_step(self, batch, batch_idx):
      inputs,labels = batch
      outputs,labels = self.step(inputs,labels)
      predictions = torch.argmax(outputs,dim=-1)
      return predictions,labels

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.learning_rate,
        )
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer=optimizer,
            factor=0.5,
            patience=1,
            verbose=True,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "val_loss",
        }

In [25]:
def train_model(
    model,
    train_dataloader,
    val_dataloader,
    text_type,
    max_epochs=50,
    validate_and_fit=False
  ):
  checkpoints_path = Path(f"./DotsRetrieval/{text_type}")
  if validate_and_fit:
    shutil.rmtree(checkpoints_path, ignore_errors=True)
  checkpoint_callback = ModelCheckpoint(
      mode="min",
      save_top_k=1,
      verbose=False,
      save_last=True,
      monitor="val_loss",
      save_weights_only=False,
      auto_insert_metric_name=True,
      save_on_train_epoch_end=False,
      dirpath=f"{checkpoints_path}/checkpoints",
      filename="{epoch}-{val_loss:.3f}-{step}",
  )
  callbacks = list()
  callbacks.append(checkpoint_callback)
  early_stopping_callback = EarlyStopping(
      monitor="val_loss",
      # min_delta=0.025,
      min_delta=0,
      patience=10,
      check_finite=True,
  )
  callbacks.append(early_stopping_callback)
  lr_monitor = LearningRateMonitor(
      logging_interval="step",
      log_momentum=True,
  )
  callbacks.append(lr_monitor)
#   callbacks.append(RichProgressBar())
  devices = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  trainer = Trainer(
      deterministic=True,
      callbacks=callbacks,
      gradient_clip_val=5,
      fast_dev_run=False,
      max_epochs=max_epochs,
      val_check_interval=0.25,
      accelerator="auto",
      devices=[1],
      # log_every_n_steps=max(len(train_dataloader) // 25, 1),
      log_every_n_steps=max(len(train_dataloader) // 25, 1),
  )
  if validate_and_fit:
    trainer.validate(
        model=model,
        dataloaders=val_dataloader,
    )
    trainer.fit(
        model,
        train_dataloader,
        val_dataloader,
  )
  return trainer

# Prepare vocab

## source tokenizer

In [26]:
source_tokenizer = tk.CharacterTokenizer(vocab_size=10_000_000)

In [27]:
source_tokenizer.train(text='\n'.join(undotted_train_dataset))

Training CharacterTokenizer ...


In [28]:
source_tokenizer.vocab_size

40

In [29]:
# test the tokenizer
source_tokenizer.tokenize(undot('السلام عليكم و رحمة الله و بركاته')),source_tokenizer.encode(undot('السلام عليكم و رحمة الله و بركاته'))

(['ا',
  '##ل',
  '##س',
  '##ل',
  '##ا',
  '##م',
  'ع',
  '##ل',
  '##ٮ',
  '##ك',
  '##م',
  'و',
  'ر',
  '##ح',
  '##م',
  '##ه',
  'ا',
  '##ل',
  '##ل',
  '##ه',
  'و',
  'ٮ',
  '##ر',
  '##ك',
  '##ا',
  '##ٮ',
  '##ه'],
 [5,
  4,
  13,
  4,
  3,
  8,
  23,
  4,
  2,
  21,
  8,
  18,
  33,
  11,
  8,
  6,
  5,
  4,
  4,
  6,
  18,
  12,
  7,
  21,
  3,
  2,
  6])

## target tokenizer

In [30]:
target_tokenizer = tk.CharacterTokenizer(vocab_size=10_000_000)

In [31]:
target_tokenizer.train(text='\n'.join(train_dataset))

Training CharacterTokenizer ...


In [32]:
target_tokenizer.vocab_size

62

In [33]:
# test the tokenizer
target_tokenizer.tokenize('السلام عليكم و رحمة الله و بركاته'),target_tokenizer.encode('السلام عليكم و رحمة الله و بركاته')

(['ا',
  '##ل',
  '##س',
  '##ل',
  '##ا',
  '##م',
  'ع',
  '##ل',
  '##ي',
  '##ك',
  '##م',
  'و',
  'ر',
  '##ح',
  '##م',
  '##ة',
  'ا',
  '##ل',
  '##ل',
  '##ه',
  'و',
  'ب',
  '##ر',
  '##ك',
  '##ا',
  '##ت',
  '##ه'],
 [5,
  3,
  15,
  3,
  2,
  7,
  26,
  3,
  4,
  20,
  7,
  16,
  49,
  22,
  7,
  11,
  5,
  3,
  3,
  18,
  16,
  24,
  8,
  20,
  2,
  9,
  18])

In [34]:
target_tokenizer.detokenize(target_tokenizer.tokenize('السلام عليكم و رحمة الله و بركاته'))

'السلام عليكم و رحمة الله و بركاته'

# Run the experiment

## tokenize and split

In [40]:
encoded_trainset = create_features_from_text_list(text_list=undotted_train_dataset,tokenizer=source_tokenizer)
trainy = create_features_from_text_list(text_list=train_dataset,tokenizer=target_tokenizer)

  0%|          | 0/1368224 [00:00<?, ?it/s]

  0%|          | 0/1368224 [00:00<?, ?it/s]

In [41]:
encoded_testset = create_features_from_text_list(text_list=undotted_test_dataset,tokenizer=source_tokenizer)
testy = create_features_from_text_list(text_list=test_dataset,tokenizer=target_tokenizer)

  0%|          | 0/72025 [00:00<?, ?it/s]

  0%|          | 0/72025 [00:00<?, ?it/s]

In [42]:
testy_with_dots = create_features_from_text_list(text_list=test_dataset,tokenizer=target_tokenizer)

  0%|          | 0/72025 [00:00<?, ?it/s]

In [43]:
encoded_trainset, encoded_valset, trainy, valy = train_test_split(
  encoded_trainset,
  trainy,
  test_size=0.1,
  random_state=seed,
)
len(encoded_trainset),len(encoded_valset),len(trainy), len(valy)

(1231401, 136823, 1231401, 136823)

In [44]:
encoded_trainset.shape,trainy.shape

((1231401, 500), (1231401, 500))

In [45]:
# create tensor datasets
trainset = TensorDataset(torch.from_numpy(encoded_trainset), torch.from_numpy(trainy))
validset = TensorDataset(torch.from_numpy(encoded_valset), torch.from_numpy(valy))
testset = TensorDataset(torch.from_numpy(encoded_testset), torch.from_numpy(testy))
testset_with_dots = TensorDataset(torch.from_numpy(encoded_testset), torch.from_numpy(testy_with_dots))

In [46]:
# create dataloaders
trainloader = DataLoader(trainset, shuffle=True, batch_size=batch_size,num_workers=4)
valloader = DataLoader(validset, shuffle=False, batch_size=batch_size,num_workers=4,drop_last=False)
testloader = DataLoader(testset, shuffle=False, batch_size=batch_size,num_workers=4,drop_last=False)
testloader_with_dots = DataLoader(testset_with_dots,shuffle=False,batch_size=batch_size,num_workers=4,drop_last=False)

## build and train the model

In [47]:
model = LitBiLSTMModel(
    vocab_size=source_tokenizer.vocab_size,
    output_size=target_tokenizer.vocab_size,
  )
model

LitBiLSTMModel(
  (train_accuracy): MulticlassAccuracy()
  (val_accuracy): MulticlassAccuracy()
  (test_accuracy): MulticlassAccuracy()
  (embedding): Embedding(40, 512)
  (lstm): LSTM(512, 512, num_layers=2, batch_first=True, dropout=0.33, bidirectional=True)
  (dropout): Dropout(p=0.33, inplace=False)
  (embedding_dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=512, out_features=62, bias=True)
)

In [48]:
trainer = train_model(
    model,
    train_dataloader=trainloader,
    val_dataloader=valloader,
    text_type='dotless-to-dotted',
  )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [49]:
trainer.test(
    model=model,
    ckpt_path='./DotsRetrieval/dotless-to-dotted/checkpoints/epoch=49-val_loss=0.030-step=279625.ckpt',
    dataloaders=testloader,
)

You are using a CUDA device ('NVIDIA RTX A4500') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at ./DotsRetrieval/dotless-to-dotted/checkpoints/epoch=49-val_loss=0.030-step=279625.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at ./DotsRetrieval/dotless-to-dotted/checkpoints/epoch=49-val_loss=0.030-step=279625.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9916979670524597     │
│         test_loss         │   0.024777263402938843    │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9916979670524597, 'test_loss': 0.024777263402938843}]

find the text metrics, (wer,cer,ver)

In [51]:
model_predictions = trainer.predict(
    model=model,
    dataloaders=testloader,
    ckpt_path='./DotsRetrieval/dotless-to-dotted/checkpoints/epoch=49-val_loss=0.030-step=279625.ckpt',
)

Restoring states from the checkpoint path at ./DotsRetrieval/dotless-to-dotted/checkpoints/epoch=49-val_loss=0.030-step=279625.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at ./DotsRetrieval/dotless-to-dotted/checkpoints/epoch=49-val_loss=0.030-step=279625.ckpt


Predicting: 0it [00:00, ?it/s]

In [52]:
predictions = list()
labels = list()
for (batch_predictions,batch_labels) in model_predictions:
  batch_predictions = batch_predictions.view(-1,seq_len)
  batch_labels = batch_labels.view(-1,seq_len)
  for sample_predictions,sample_labels in zip(batch_predictions,batch_labels):
    predictions.append(sample_predictions)
    labels.append(sample_labels)
print('len predictions and lables of the test set:',len(predictions),len(labels))

len predictions and lables of the test set: 72025 72025


In [97]:
wers,cers,vers = list(),list(),list()
for sample_preds,sample_labels in tqdm(zip(predictions[:100],labels[:100]),total=len(predictions)):
  wer,cer,p,t = calculate_text_metrics(
      predictions=sample_preds.tolist(),
      labels=sample_labels.tolist(),
      target_tokenizer=target_tokenizer,
      print_text=True,
    )
  wers.append(wer)
  cers.append(cer)

  0%|          | 0/72025 [00:00<?, ?it/s]

****************************************************************************************************
في حزيران غادر ليون بلوم منصبه وعاد ادوار دالادير الا الحكم رافعا شعار التمسك بالامبراطورية الفرنسية فسافر الرءيس جميل مردم بك الا باريس في تشرين الثاني ووقع عدة ملاحق للمعاهدة السورية الفرنسية فيها الكثير من التنازلات منها ضمانات اضافية للاقليات والتاكيد علا الاستعانة بخبراء فرنسيين داءمين في مءسسات الحكم السورية
في حزيران غادر ليون بلوم منصبه وعاد ادوار دالادير الا الحكم رافعا شعار التمسك بالامبراطورية الفرنسية فسافر الرءيس جميل مردم بك الا باريس في تشرين الثاني ووقع عدة ملاحق للمعاهدة السورية الفرنسية فيها الكثير من التنازلات منها ضمانات اضافية للاقليات والتاكيد علا الاستعانة بخبراء فرنسيين داءمين في مءسسات الحكم السورية
****************************************************************************************************
يعتبر الحكم سعد العقيل من الحكام السابقين في التحكيم ولقد عاصر كثيرا من الحكام مثل الحكم عمر المهنا وعبد الله الناصر وفلاح الستار والشريف وعبد الرحمن الدهام وفهد الده

/home/majed_alshaibani/Experiments/dotless-arabic/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:70: FutureWarning: Importing `word_error_rate` from `torchmetrics.functional` was deprecated and will be removed in 2.0. Import `word_error_rate` from `torchmetrics.text` instead.
  _future_warning(
/home/majed_alshaibani/Experiments/dotless-arabic/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:70: FutureWarning: Importing `char_error_rate` from `torchmetrics.functional` was deprecated and will be removed in 2.0. Import `char_error_rate` from `torchmetrics.text` instead.
  _future_warning(


****************************************************************************************************
كما يعتبر اتحاد موردي المياه والطاقة الالماني وهو مجموعة ضغط نشطة في مجال ادارة الطاقة ان الانتقال الطاقي امرا لا رجعة فيه ففي ابريل صرح هيلديغارد مولر المديرة الرءيسية للاتحاد لصحيفة فدي باكريشين الاسبوعية قاءلة لم يعد الامر يتعلق بمعرفته فقط بل كيفية تنفيذ الانتقال ودعت في الوقت نفسه الا توجيه افضل من قبل المسءولين السياسيين للتجول في سوق الطاقة واخيرا وضع المواطنون انفسهم كجهات نشطة في العملية يمتلكون من الطاقة الانتاجية للطاقة المتجددة من خلال التعاون
كما يعتبر اتحاد موردي المياه والطاقة الالماني وهو مجموعة ضغط نشطة في مجال ادارة الطاقة ان الانتقال الطاقي امرا لا رجعة فيه ففي ابريل صرح هيلديغارد مولر المديرة الرءيسية للاتحاد لصحيفة فدي ناكريشتن الاسبوعية قاءلة لم يعد الامر يتعلق بمعرفته فقط بل كيفية تنفيذ الانتقال ودعت في الوقت نفسه الا توجيه افضل من قبل المسءولين السياسيين للتحول في سوق الطاقة واخيرا وضع المواطنون انفسهم كجهات نشطة في العملية يمتلكون من الطاقة الانتاجية للطاقة المت

In [98]:
avg_wer = sum(wers)/len(wers)
avg_cer = sum(cers)/len(cers)
print('avg_wer, avg_cer:',avg_wer,avg_cer)

avg_wer, avg_cer: tensor(0.0316) tensor(0.0072)


text metrics with vowel words loader

In [ ]:
labels_with_dots = list()
for batch in testloader_with_dots:
  _,batch_labels = batch
  batch_labels = batch_labels.view(-1,seq_len)
  for sample_labels in batch_labels:
    labels_with_dots.append(sample_labels)
print('len lables with vowel words of the test set:',len(labels_with_dots))

In [ ]:
wers,cers = list(),list()
for sample_preds,sample_labels in tqdm(zip(predictions,labels_with_dots),total=len(predictions)):
  wer,cer,ver = calculate_text_metrics(
      predictions=sample_preds.tolist(),
      labels=sample_labels.tolist(),
      target_tokenizer=target_tokenizer,
    )
  wers.append(wer)
  cers.append(cer)

In [ ]:
avg_wer = sum(wers)/len(wers)
avg_cer = sum(cers)/len(cers)
print('avg_wer, avg_cer, avg_ver:',avg_wer,avg_cer)

test on the best model according to the validation loss

In [ ]:
# model = LitBiLSTMModel.load_from_checkpoint(
#     trainer.checkpoint_callback.best_model_path,
#     vocab_size=source_tokenizer.vocab_size,
#     output_size=target_tokenizer.vocab_size,
#   )
# model

In [ ]:
# trainer.test(model,testloader)

In [ ]:
# model_predictions = trainer.predict(model,testloader)

In [ ]:
# predictions = list()
# labels = list()
# for (batch_predictions,batch_labels) in model_predictions:
#   batch_predictions = batch_predictions.view(-1,seq_len)
#   batch_labels = batch_labels.view(-1,seq_len)
#   for sample_predictions,sample_labels in zip(batch_predictions,batch_labels):
#     predictions.append(sample_predictions)
#     labels.append(sample_labels)
# print('len predictions and lables of the test set:',len(predictions),len(labels))

In [ ]:
# wers,cers,vers = list(),list(),list()
# for sample_preds,sample_labels in tqdm(zip(predictions,labels),total=len(predictions)):
#   wer,cer,ver = calculate_text_metrics(
#       predictions=sample_preds.tolist(),
#       labels=sample_labels.tolist(),
#       target_tokenizer=target_tokenizer
#     )
#   wers.append(wer)
#   cers.append(cer)
#   vers.append(ver)

In [ ]:
# avg_wer = sum(wers)/len(wers)
# avg_cer = sum(cers)/len(cers)
# avg_ver = sum(vers)/len(vers)
# print('avg_wer, avg_cer, avg_ver:',avg_wer,avg_cer,avg_ver)

text metrics with vowel words

In [ ]:
# labels_with_vowel_words = list()
# for batch in testloader_with_vowel_words:
#   _,batch_labels = batch
#   batch_labels = batch_labels.view(-1,seq_len)
#   for sample_labels in batch_labels:
#     labels_with_vowel_words.append(sample_labels)
# print('len lables with vowel words of the test set:',len(labels_with_vowel_words))

In [ ]:
# wers,cers,vers = list(),list(),list()
# for sample_preds,sample_labels in tqdm(zip(predictions,labels_with_vowel_words),total=len(predictions)):
#   wer,cer,ver = calculate_text_metrics(
#       predictions=sample_preds.tolist(),
#       labels=sample_labels.tolist(),
#       target_tokenizer=target_tokenizer,
#     )
#   wers.append(wer)
#   cers.append(cer)
#   vers.append(ver)

In [ ]:
# avg_wer = sum(wers)/len(wers)
# avg_cer = sum(cers)/len(cers)
# avg_ver = sum(vers)/len(vers)
# print('avg_wer, avg_cer, avg_ver:',avg_wer,avg_cer,avg_ver)

In [ ]:
# for sample_preds,sample_labels in tqdm(list(zip(predictions,labels_with_vowel_words))[:100],total=100):
#   wer,cer,ver = calculate_text_metrics(
#       predictions=sample_preds.tolist(),
#       labels=sample_labels.tolist(),
#       target_tokenizer=target_tokenizer,
#       print_text=True
#     )

In [ ]:
# from google.colab import runtime
# runtime.unassign()